<a href="https://colab.research.google.com/github/vnavya2004/BTP/blob/main/Paper(A_Prompt_Based_Topic_Modeling_Method_for_Depression_Detection_on_Low_Resource_Data)_Spanish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install transformers openpyxl

# Import libraries
import pandas as pd
import torch
from transformers import BertTokenizer, BertForNextSentencePrediction
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import random
from tqdm import tqdm
import re

# Load the dataset
df = pd.read_excel('spanish.xlsx')  # Replace with your file path
df = df[['Tweets_english', 'Labels']]  # Ensure columns 'text' and 'label' exist

# Separate the data into two DataFrames based on the label
df_label_0 = df[df['Labels'] == 0]
df_label_1 = df[df['Labels'] == 1]

# Find the minimum count between the two labels to balance the dataset
min_count = min(len(df_label_0), len(df_label_1))

# Sample equal rows from each label
df_balanced = pd.concat([df_label_0.sample(n=min_count, random_state=42),
                         df_label_1.sample(n=min_count, random_state=42)], ignore_index=True)

# Shuffle the balanced dataset (optional)
df = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Sentence Segmentation function
def segment_text_by_sentences(text):
    # Use regex to split text based on sentence-ending punctuation (e.g., period, exclamation mark, question mark)
    sentences = re.split(r'(?<=[.!?]) +', text.strip())  # Split by period, exclamation mark or question mark followed by space
    return [sentence for sentence in sentences if sentence]

# Updated prompt-based emotion detection function (with random prompt selection)
def prompt_emotion(text, tokenizer, model):
    # Define the extended positive and negative prompt lists
    positive_prompts = [
        "I am very good", "I am very joyful", "I am very positive",
        "I am feeling good", "I am feeling joyful", "I am feeling positive",
        "It makes me good", "It makes me joyful", "It makes me positive",
        "That sounds good", "That sounds joyful", "That sounds positive"
    ]

    negative_prompts = [
        "I am very depressed", "I am very sad", "I am very negative",
        "I am feeling bad", "I am feeling depressed", "I am feeling sad",
        "It makes me sad", "It makes me depressed", "It makes me negative",
        "That sounds bad", "That sounds depressed", "That sounds sad"
    ]

    # Randomly choose one positive and one negative prompt
    positive_prompt = random.choice(positive_prompts)
    negative_prompt = random.choice(negative_prompts)

    # Tokenize inputs
    inputs_pos = tokenizer(text, positive_prompt, return_tensors='pt', truncation=True).to(device)
    inputs_neg = tokenizer(text, negative_prompt, return_tensors='pt', truncation=True).to(device)

    # Predict scores for both positive and negative prompts
    output_pos = model(**inputs_pos).logits
    output_neg = model(**inputs_neg).logits

    # Calculate probabilities
    prob_pos = torch.softmax(output_pos, dim=1)[0][0].item()
    prob_neg = torch.softmax(output_neg, dim=1)[0][0].item()

    # Return emotion score (higher negative score indicates depression)
    return prob_neg - prob_pos

# Zero Shot Setting - Calculate emotion score and average over sentence segments
def zero_shot_prediction(sentences, tokenizer, model):
    segment_scores = []

    # Get emotion score for each sentence
    for sentence in sentences:
        segment_score = prompt_emotion(sentence, tokenizer, model)
        segment_scores.append(segment_score)

    # Calculate the average emotion score across sentences
    avg_score = sum(segment_scores) / len(segment_scores)

    # Return the final prediction based on average score
    return 1 if avg_score < 0 else 0  # 1 = depressed, 0 = not depressed

# Fusion Model for Few-Shot Learning (One-Shot)
class FusionModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=32):
        super(FusionModel, self).__init__()

        # First fully connected layer
        self.fc1 = nn.Linear(input_dim, hidden_dim)

        # Second fully connected layer
        self.fc2 = nn.Linear(hidden_dim, 1)

        # ReLU activation for the first layer
        self.relu = nn.ReLU()

    def forward(self, x):
        # Pass the input through the first fully connected layer and apply ReLU
        x = self.relu(self.fc1(x))

        # Pass the output through the second fully connected layer
        x = self.fc2(x)

        # Apply Sigmoid to get probabilities
        return torch.sigmoid(x)


# Zero-Shot Evaluation Function
def evaluate_zero_shot(df, tokenizer, model):
    # Segment text into sentences first
    df['sentences'] = df['Tweets_english'].progress_apply(segment_text_by_sentences)

    # Calculate predictions for all entries in zero-shot setting
    df['prediction_zero_shot'] = df['sentences'].progress_apply(lambda x: zero_shot_prediction(x, tokenizer, model))

    # Calculate metrics
    y_true = df['Labels']
    y_pred = df['prediction_zero_shot']
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print("Zero-Shot Evaluation Results:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print("")

# Few-Shot Evaluation (One-Shot) - Segmenting text by sentences
def evaluate_one_shot(df, tokenizer, model):
    # Segment the input text and get emotion scores for each sentence
    all_scores = []

    # Loop over all tweets and segment them into sentences
    for sentences in df['sentences']:
        segment_scores = []

        for sentence in sentences:
            # For each sentence, calculate emotion score
            segment_score = prompt_emotion(sentence, tokenizer, model)
            segment_scores.append(segment_score)

        # Calculate the average score for the tweet (one-shot learning)
        avg_score = sum(segment_scores) / len(segment_scores)
        all_scores.append(avg_score)

    # Convert scores into tensor format
    X = torch.tensor(all_scores).float().view(-1, 1).to(device)
    y = torch.tensor(df['Labels'].values).float().view(-1, 1).to(device)

    # Split into onefold train, validation, and test sets
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.9, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.89, random_state=42)

    # Initialize and train FusionModel for adaptive voting
    fusion_model = FusionModel(1).to(device)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(fusion_model.parameters(), lr=0.001)
    epochs = 10

    # Training loop with validation after each epoch
    for epoch in range(epochs):
        fusion_model.train()
        optimizer.zero_grad()

        # Training forward pass
        outputs = fusion_model(X_train)
        loss = criterion(outputs, y_train)

        # Backpropagation
        loss.backward()
        optimizer.step()

        # Validation after each epoch
        fusion_model.eval()  # Set the model to evaluation mode
        with torch.no_grad():
            # Validate on validation set
            val_outputs = fusion_model(X_val)
            val_loss = criterion(val_outputs, y_val)
            val_pred = (val_outputs >= 0.5).float()
            val_accuracy = accuracy_score(y_val.cpu(), val_pred.cpu())
            val_precision = precision_score(y_val.cpu(), val_pred.cpu())
            val_recall = recall_score(y_val.cpu(), val_pred.cpu())
            val_f1 = f1_score(y_val.cpu(), val_pred.cpu())

        # Print the validation results after each epoch
        print(f"Epoch [{epoch+1}/{epochs}], Training Loss: {loss.item():.4f}")
        print(f"Validation Loss: {val_loss.item():.4f}")
        print(f"Validation Accuracy: {val_accuracy:.4f}")
        print(f"Validation Precision: {val_precision:.4f}")
        print(f"Validation Recall: {val_recall:.4f}")
        print(f"Validation F1-Score: {val_f1:.4f}\n")

    # Test after training
    fusion_model.eval()
    with torch.no_grad():
        test_outputs = fusion_model(X_test)
        test_pred = (test_outputs >= 0.5).float()
        test_accuracy = accuracy_score(y_test.cpu(), test_pred.cpu())
        test_precision = precision_score(y_test.cpu(), test_pred.cpu())
        test_recall = recall_score(y_test.cpu(), test_pred.cpu())
        test_f1 = f1_score(y_test.cpu(), test_pred.cpu())

    print("\nTest Results:")
    print(f"Accuracy: {test_accuracy:.4f}")
    print(f"Precision: {test_precision:.4f}")
    print(f"Recall: {test_recall:.4f}")
    print(f"F1-Score: {test_f1:.4f}")

# List of PLMs to test in zero-shot setting
plm_names = [
    "bert-base-uncased"
]



In [ ]:
plm_name="bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(plm_name)
model = BertForNextSentencePrediction.from_pretrained(plm_name).to(device)
tqdm.pandas()
# Zero-Shot Evaluation
evaluate_zero_shot(df, tokenizer, model)


In [3]:
# Few-Shot Evaluation (One-Shot)
evaluate_one_shot(df, tokenizer, model)

Epoch [1/10], Training Loss: 0.7392
Validation Loss: 0.7008
Validation Accuracy: 0.4351
Validation Precision: 0.4813
Validation Recall: 0.5485
Validation F1-Score: 0.5127

Epoch [2/10], Training Loss: 0.7384
Validation Loss: 0.7076
Validation Accuracy: 0.5021
Validation Precision: 0.4861
Validation Recall: 0.5529
Validation F1-Score: 0.5174

Epoch [3/10], Training Loss: 0.7376
Validation Loss: 0.7041
Validation Accuracy: 0.5551
Validation Precision: 0.5090
Validation Recall: 0.5630
Validation F1-Score: 0.5346

Epoch [4/10], Training Loss: 0.7369
Validation Loss: 0.7040
Validation Accuracy: 0.4733
Validation Precision: 0.5036
Validation Recall: 0.5537
Validation F1-Score: 0.5275

Epoch [5/10], Training Loss: 0.7361
Validation Loss: 0.7035
Validation Accuracy: 0.5253
Validation Precision: 0.5300
Validation Recall: 0.5738
Validation F1-Score: 0.5510

Epoch [6/10], Training Loss: 0.7354
Validation Loss: 0.7009
Validation Accuracy: 0.4999
Validation Precision: 0.5479
Validation Recall: 0.59